In [162]:
import pandas as pd

In [163]:
df_499 = pd.read_pickle("/Users/Mikhail_Bulgakov/GitRepo/jdi-qasp-ml/data/mui_dataset/df/site-499.pkl")

In [164]:
df_499.head()

,attributes,displayed,element_id,height,onmouseenter,onmouseover,parent_id,style,tag_name,text,width,x,y
0,{'lang': 'en'},True,2896533131556138215791681852,6565.828125,None,None,None,"[accent-color, align-content, align-items, ali...",HTML,Inbox\nStarred\nSend email\nDrafts\nAll mail\n...,1227.0,0.0,0.0
1,None,False,3247971000556138217773112066,0.000000,None,None,2896533131556138215791681852,"[accent-color, align-content, align-items, ali...",HEAD,React App\n.MuiSvgIcon-root {\n fill: current...,0.0,0.0,0.0
2,{'charset': 'utf-8'},False,3198177535556138217774820701,0.000000,None,None,3247971000556138217773112066,"[accent-color, align-content, align-items, ali...",META,,0.0,0.0,0.0
3,"{'href': './favicon.ico', 'rel': 'icon'}",False,5441131729556138213937558124,0.000000,None,None,3247971000556138217773112066,"[accent-color, align-content, align-items, ali...",LINK,,0.0,0.0,0.0
4,"{'content': 'width=device-width,initial-scale=...",False,5857506368556138218104768089,0.000000,None,None,3247971000556138217773112066,"[accent-color, align-content, align-items, ali...",META,,0.0,0.0,0.0


In [165]:
df_499['attr_keys'] = df_499.attributes.apply(lambda x: [None] if x is None else list(x.keys()))

In [217]:
# df_499_labeled = df_499[df_499.attributes.astype(str).str.contains("data-label")].reset_index(drop=True)
df_499_labeled = df_499.copy()
df_499_labeled['STRING'] = df_499_labeled.attributes.apply((lambda x: None if x is None else x.get('class', "") + " " + x.get('type', "") + " " + x.get('role', "")))
df_499_labeled['STRING'] = df_499_labeled.tag_name.str.cat(df_499_labeled['STRING'], sep=" ")
df_499_labeled['LABEL'] = df_499_labeled.attributes.apply(lambda x: None if x is None else x.get('data-label'))
df_499_labeled = df_499_labeled[["element_id", "parent_id", "STRING", "LABEL"]]
df_499_labeled.reset_index(drop=True, inplace=True)

In [218]:
import itertools
par_childs = {par:list(df_499_labeled[df_499_labeled.parent_id==par].element_id.unique()) for par in df_499_labeled.parent_id.unique()}

def find_all_childs(elem_id):
    # print(df_499[df_499.element_id==elem_id].attributes.iloc[0])
    if par_childs.get(elem_id) is None:
        return []
    else:
        res = []
        for i in par_childs.get(elem_id):
            res.append(i)
            res.extend(find_all_childs(i))
        return res

In [219]:
# find_all_childs("7416253826556138227472466804")

In [220]:
df_499_labeled

,element_id,parent_id,STRING,LABEL
0,2896533131556138215791681852,None,HTML,None
1,3247971000556138217773112066,2896533131556138215791681852,NaN,None
2,3198177535556138217774820701,3247971000556138217773112066,META,None
3,5441131729556138213937558124,3247971000556138217773112066,LINK,None
4,5857506368556138218104768089,3247971000556138217773112066,META,None
...,...,...,...,...
1122,1027648080556138223518938445,4303492041556138227684109879,P MuiTypography-root jss85 MuiTypography-body1,typography
1123,1999577821556138222703290190,9532308721556138227446723085,DIV,None
1124,3663659299556138222210445554,9301401088556138216135889332,NaN,None
1125,2458135316556138223806734564,9301401088556138216135889332,DIV tooltip,popper


In [221]:
df_499_labeled['all_followers'] = df_499_labeled['element_id'].apply(find_all_childs)
df_499_labeled.head()

,element_id,parent_id,STRING,LABEL,all_followers
0,2896533131556138215791681852,None,HTML,None,"[3247971000556138217773112066, 319817753555613..."
1,3247971000556138217773112066,2896533131556138215791681852,NaN,None,"[3198177535556138217774820701, 544113172955613..."
2,3198177535556138217774820701,3247971000556138217773112066,META,None,[]
3,5441131729556138213937558124,3247971000556138217773112066,LINK,None,[]
4,5857506368556138218104768089,3247971000556138217773112066,META,None,[]


In [222]:
elem_string_dict = {k:v.get('STRING') for k,v in df_499_labeled[['STRING']].set_index(df_499_labeled.element_id).fillna('').T.to_dict().items()}
elem_string_dict

{'2896533131556138215791681852': 'HTML   ',
 '3247971000556138217773112066': '',
 '3198177535556138217774820701': 'META   ',
 '5441131729556138213937558124': 'LINK   ',
 '5857506368556138218104768089': 'META   ',
 '3083372211556138216318296343': 'META   ',
 '8817921946556138213546688681': 'META   ',
 '3478858333556138210966547855': 'LINK   ',
 '6232503495556138210565423412': 'LINK   ',
 '4659777762556138214354854925': '',
 '8791768518556138211000023595': 'STYLE   ',
 '6706239372556138212158464814': 'STYLE   ',
 '0753565503556138211235297003': 'STYLE   ',
 '2484548330556138219481129435': 'STYLE   ',
 '3191637529556138215684213873': 'STYLE   ',
 '9361576063556138212569482624': 'STYLE   ',
 '1389362281556138211888531828': 'STYLE   ',
 '2975651472556138215403613998': 'STYLE   ',
 '1355904214556138219259938327': 'STYLE   ',
 '2682792595556138211074409122': 'STYLE   ',
 '7608793517556138213836390653': 'STYLE   ',
 '9366923939556138210511516932': 'STYLE   ',
 '0413168558556138210902887660': '

In [223]:

def concat_all_strings(list):
    res = ''
    for l in list:
        res = res + elem_string_dict.get(l) + ' '
    return res

In [224]:
df_499_labeled['all_followers_string'] = df_499_labeled['all_followers'].apply(concat_all_strings)
df_499_labeled['final_string'] = df_499_labeled.STRING + ' ' + df_499_labeled.all_followers_string

In [268]:
dataset = df_499_labeled[["element_id", "final_string", "LABEL"]].copy()
dataset.final_string.fillna("", inplace=True)
dataset

,element_id,final_string,LABEL
0,2896533131556138215791681852,HTML META LINK META META META ...,None
1,3247971000556138217773112066,,None
2,3198177535556138217774820701,META,None
3,5441131729556138213937558124,LINK,None
4,5857506368556138218104768089,META,None
...,...,...,...
1122,1027648080556138223518938445,P MuiTypography-root jss85 MuiTypography-body1,typography
1123,1999577821556138222703290190,DIV,None
1124,3663659299556138222210445554,,None
1125,2458135316556138223806734564,DIV tooltip DIV jss57,popper


In [269]:
# df_499_labeled[df_499_labeled.element_id=="7416253826556138227472466804"]

In [270]:
# test.loc[953]

In [271]:
# for i in test.loc[953]:
#     print(df_499_labeled[df_499_labeled.element_id==i].STRING.iloc[0])

In [272]:
import re
mystring = "div muibuttonbase root muilistitem root"
re.sub(' +', ' ', re.sub('(root)|(mui)', ' ', mystring).strip())

'div buttonbase listitem'

In [273]:

def prepare_string(string):
    string = str(string).lower().strip()
    return(re.sub(' +', ' ', re.sub('(root)|(mui)|(-)|([0-9]+)', ' ', string).strip()))



dataset['final_string'] = dataset['final_string'].apply(prepare_string)
dataset[~dataset.LABEL.isna()]

,element_id,final_string,LABEL
120,6932874036556138214282815018,div drawer drawer docked jss div paper drawer ...,drawer
123,1020730951556138214617022005,hr divider,divider
124,1485373126556138215821397030,ul list list padding div buttonbase listitem l...,list
125,7619432732556138211657671930,div buttonbase listitem listitem gutters listi...,button
127,5006822130556138215329088562,svg svgicon path,icon
...,...,...,...
1116,2997734401556138224550088524,div paper paper elevation paper rounded p typo...,paper
1117,7996883338556138223728211052,p typography jss typography body,typography
1118,9532308721556138227446723085,div popover presentation div div div paper pop...,popover
1122,1027648080556138223518938445,p typography jss typography body,typography


In [274]:
buttons = dataset[(dataset.LABEL=='button')&(~dataset.LABEL.isna())]
buttons

,element_id,final_string,LABEL
125,7619432732556138211657671930,div buttonbase listitem listitem gutters listi...,button
132,0544872677556138210457419249,div buttonbase listitem listitem gutters listi...,button
139,7720998232556138215373165521,div buttonbase listitem listitem gutters listi...,button
146,1230668619556138210299365813,div buttonbase listitem listitem gutters listi...,button
155,7197951295556138213903836911,div buttonbase listitem listitem gutters listi...,button
162,8997295726556138213338468420,div buttonbase listitem listitem gutters listi...,button
169,5054340968556138218032572542,div buttonbase listitem listitem gutters listi...,button
193,4754430374556138211174729089,button buttonbase iconbutton jss iconbutton co...,button
222,7414575006556138212945950473,button buttonbase iconbutton iconbutton colori...,button
229,8668341540556138218450065148,button buttonbase iconbutton iconbutton colori...,button


In [275]:
buttons = dataset[(dataset.LABEL=='icon')&(~dataset.LABEL.isna())]
buttons

,element_id,final_string,LABEL
127,5006822130556138215329088562,svg svgicon path,icon
134,5653779534556138218241979291,svg svgicon path,icon
141,2245620486556138214586731389,svg svgicon path,icon
148,5805904440556138217684878647,svg svgicon path,icon
157,0831200404556138210314094863,svg svgicon path,icon
164,2479693400556138217062455857,svg svgicon path,icon
171,9572435876556138219469158226,svg svgicon path,icon
178,2500082132556138219398607748,svg svgicon path,icon
180,3020214565556138219248841495,svg svgicon path,icon
182,8290977288556138211745058228,svg svgicon path,icon


In [ ]:
buttons = dataset[(dataset.LABEL=='button')&(~dataset.LABEL.isna())]
buttons